In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# define constants
d1 = 0.025  # m # delta_1
d2 = 0.050  # m # delta_2

R1 = 0.025  # m
R2 = R1 + d1  # m
R3 = R2 + d2  # m

rho1 = 2600  # kg/m^3
cp1 = 1150  # J/kg-K
k1 = 3.0  # W/m-K

alpha1 = k1 / (rho1 * cp1)  # m^2/s

rho2 = 600  # kg/m^3
cp2 = 200  # J/kg-K
k2 = 0.2  # W/m-K

alpha2 = k2 / (rho2 * cp2)  # m^2/s

T_init = 300  # K
T_0 = 1500  # K
T_inf = 298  # K

h1 = 500  # W/m^2-K
h2 = 10  # W/m^2-K

In [3]:
def calc_first(_Ti: float, _dr: float) -> float:
  """
  _Ti: temperature of T[1]
  _dr: delta r
  """
  return (h1 * T_0 + k1 * _Ti / _dr) / (h1 + k1 / _dr)


def calc(_Tl: float, _Ti: float, _Tr: float, _dr: float, _alpha: float, _dt: float, _i: int) -> float:
  """
  _Tl: temperature of T[i - 1]
  _Ti: temperature of T[i]
  _Tr: temperature of T[i + 1]
  _dr: delta r
  _alpha: alpha
  _dt: delta t
  _i: ith point
  """
  ri_ = R1 + _i * _dr  # r of T[i]
  return _Ti + _alpha * _dt * ((_Tl + _Tr - 2 * _Ti) / _dr**2 + (1 / ri_) * (_Tr - _Tl) / (2 * _dr))


def calc_middle(_Tl: float, _Tr: float) -> float:
  """
  _Tl: temperature of T[i - 1]
  _Tr: temperature of T[i + 1]
  """
  return (_Tl * k1 + _Tr * k2) / (k1 + k2)


def calc_last(_Ti: float, _dr: float) -> float:
  """
  _Ti: temperature of T[n - 1]
  _dr: delta r
  """
  return (h2 * T_inf + k2 * _Ti / _dr) / (h2 + k2 / _dr)

In [19]:
def iter_once(_T: np.ndarray, _n: int, _dr: float, _dt: float, _bp: int) -> np.ndarray:
  """
  _T: temperature array, n + 1 nodes
  _n: number of segments, n + 1 nodes
  _dr: delta r
  _dt: delta t
  _bp: breakpoint n of R2
  """
  T_ = np.zeros(_n + 1, dtype=np.float64)

  T_[0] = calc_first(_T[1], _dr)

  for i in range(1, _bp):
    T_[i] = calc(_T[i - 1], _T[i], _T[i + 1], _dr, alpha1, _dt, i)

  T_[_bp] = calc_middle(_T[_bp - 1], _T[_bp + 1])

  for i in range(_bp + 1, _n):
    T_[i] = calc(_T[i - 1], _T[i], _T[i + 1], _dr, alpha2, _dt, i)

  T_[_n] = calc_last(_T[_n - 1], _dr)

  return T_


def do_iter(_n: int, _dt: float, _eps: float, _d2=d2, hook_data=True, count=False) -> np.ndarray:
  T_ = np.full(_n + 1, T_init, dtype=np.float64)
  dr_ = (d1 + _d2) / _n

  bp_ = int(_n * d1 / (d1 + _d2))

  if hook_data:
    data_hook_ = []
    data_hook_.append(T_.copy())

    pass

  counter_ = 0

  while True:
    T_new = iter_once(T_, _n, dr_, _dt, bp_)

    counter_ += 1

    if hook_data:
      data_hook_.append(T_new.copy())

    if np.max(np.abs(T_new - T_)) < _eps:
      break
    T_ = T_new

  if hook_data:
    return T_new, np.array(data_hook_)

  if count:
    return T_new, counter_

  return T_new


def do_iter_l(_n: int, _dt: float, _d2=d2, count=False) -> np.ndarray:
  T_ = np.full(_n + 1, T_init, dtype=np.float64)
  dr_ = (d1 + _d2) / _n

  bp_ = int(_n * d1 / (d1 + _d2))

  counter_ = 0

  while True:
    T_new = iter_once(T_, _n, dr_, _dt, bp_)

    counter_ += 1

    if counter_ >= 400000:
      break
    T_ = T_new

  if count:
    return T_new, counter_

  return T_new


def do_iter_r(_n: int, _dt: float, _eps: float, _d2s: np.ndarray) -> np.ndarray:
  data_hook_ = []
  time_hook_ = []

  for d2_ in _d2s:
    T_, counter_ = do_iter(_n, _dt, _eps, d2_, False, True)
    data_hook_.append(T_.copy())
    time_hook_.append(counter_ * _dt)

    pass

  return np.array(data_hook_), np.array(time_hook_)


def do_iter_rl(_n: int, _dt: float, _d2s: np.ndarray) -> np.ndarray:
  data_hook_ = []
  time_hook_ = []

  for d2_ in _d2s:
    T_, counter_ = do_iter_l(_n, _dt, d2_, True)
    data_hook_.append(T_.copy())
    time_hook_.append(counter_ * _dt)

    pass

  return np.array(data_hook_), np.array(time_hook_)

In [5]:
n = 300
dt = 1e-2
eps = 1e-5

In [6]:
# plot 3D T(r, t)
def plot_3d(_T: np.ndarray):
  fig_ = plt.figure()
  ax_ = fig_.add_subplot(111, projection='3d')

  r_ = np.linspace(R1, R3, n + 1)
  iter_times_ = len(_T)
  t_all_ = iter_times_ * dt
  t_ = np.linspace(0, t_all_, iter_times_)

  r_, t_ = np.meshgrid(r_, t_)

  ax_.plot_surface(r_, t_, _T, cmap='viridis')

  ax_.set_xlabel('r (m)')
  ax_.set_ylabel('t (s)')
  ax_.set_zlabel('T (K)')
  ax_.set_title('Temperature Distribution')
  plt.savefig('../_img/T_all.png')
  plt.show()

  pass


# plot 2D T(r)
def plot_2d(_T: np.ndarray, _d2=d2, saveAs=True):
  plt.figure()
  plt.plot(np.linspace(R1, R2 + _d2, n + 1), _T)
  plt.xlabel('r (m)')
  plt.ylabel('T (K)')
  plt.title('Final Temperature distribution')
  plt.grid()
  if saveAs:
    plt.savefig(f'../_img/T_d2_{_d2:.3f}.png')
  else:
    plt.savefig('../_img/T.png')
  plt.show()

  pass


# plot 3D T(r, delta 2)
def plot_3d_r(_T: np.ndarray, _d2s: np.ndarray):
  fig_ = plt.figure()
  ax_ = fig_.add_subplot(111, projection='3d')

  for i_, T_r_ in enumerate(_T):
    d2_ = _d2s[i_]
    r_ = np.linspace(R1, R2 + d2_, n + 1)

    ax_.plot(r_, T_r_, zs=d2_, zdir='y')

  ax_.set_xlabel('r (m)')
  ax_.set_ylabel('delta 2 (m)')
  ax_.set_zlabel('T (K)')
  ax_.set_title('Final Temperature Distribution')
  plt.savefig('../_img/T_r.png')
  plt.show()

  pass

In [22]:
T, T_all = do_iter(n, dt, eps)

In [ ]:
bp_n = int(n * d1 / (d1 + d2))

T_r2 = T_all[:, bp_n]
T_r3 = T_all[:, -1]

plt.figure()
plt.plot(np.arange(len(T_r2)) * dt, T_r2, label='T at R2')
plt.plot(np.arange(len(T_r3)) * dt, T_r3, label='T at R3')
plt.xlabel('t (s)')
plt.ylabel('T (K)')
plt.title('Temperature at R2 and R3 by time')
plt.grid()
plt.legend()
plt.savefig('../_img/T_t.png')
plt.show()

plot_3d(T_all)
plot_2d(T, saveAs=False)

In [16]:
d2s = np.array([0.03, 0.3])
T_r, time_r = do_iter_r(n, dt, eps, d2s)

In [ ]:
plot_3d_r(T_r, d2s)

In [ ]:
for i, T_ in enumerate(T_r):
  plot_2d(T_, d2s[i])

  pass

In [ ]:
bps = np.array([int(n * d1 / (d1 + d2)) for d2 in d2s])


def get_eta(_Tr: float, _Tl: float, _Tenv=T_inf) -> float:
  return (_Tr - _Tenv) / (_Tl - _Tenv)


with open('../assets/data_eta.csv', 'w') as f:
  for i, T_ in enumerate(T_r):
    eta = get_eta(T_[bps[i]], T_[0])
    time = time_r[i]
    f.write(f'{d2s[i]:.3f},{eta:.4f},{time:.2f}\n')

    pass
  pass

for i, T_ in enumerate(T_r):
  eta = get_eta(T_[bps[i]], T_[0])
  print(f'delta 2: {d2s[i]:.3f}, eta: {eta:.4f}, time: {time_r[i]:.2f}')

  pass

In [20]:
T_r, time_r = do_iter_rl(n, dt, d2s)

In [ ]:
bps = np.array([int(n * d1 / (d1 + d2)) for d2 in d2s])


def get_eta(_Tr: float, _Tl: float, _Tenv=T_inf) -> float:
  return (_Tr - _Tenv) / (_Tl - _Tenv)


with open('../assets/data_eta2.csv', 'w') as f:
  for i, T_ in enumerate(T_r):
    eta = get_eta(T_[bps[i]], T_[0])
    time = time_r[i]
    f.write(f'{d2s[i]:.3f},{eta:.4f},{time:.2f}\n')

    pass
  pass

for i, T_ in enumerate(T_r):
  eta = get_eta(T_[bps[i]], T_[0])
  print(f'delta 2: {d2s[i]:.3f}, eta: {eta:.4f}, time: {time_r[i]:.2f}')

  pass